<a href="https://colab.research.google.com/github/bigyunicorn/fastai22/blob/main/lesson05/Linear_model_and_neural_net_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background
I reorganized Jeremy's Notebook for [FastAI lesson05](https://www.kaggle.com/code/bigyunicorn/linear-model-and-neural-net-from-scratch/edit) so that my future self can easily recall the content and leverage it in the future.

I am mainly focusing on the high level key points from the notes. All the code is directly coming from the Jeremy's notebook.

___
#### Notes:
I wanted to use the coding aspect leveraing an AI coding tool such as copilot or codey. Unfortunately, I do not have the access to the GPU supported GitHub Codespce or the access to Codey in Google Colab as of now (06/21/23).

I am planning to try it once either of the feature is available to me. I've tried creating [my notebook](https://github.com/bigyunicorn/fastai22/blob/main/lesson03/neural-network-from-scratch.ipynb) from the scratch by using github copilot for FastAI lesson 03. It was very helpful as I could focus on the essense of each notebook that Jeremy goes over in the lecture without worrying about the syntax too much.

I highly recommend trying this method out to others if you have access.
___
#### Tips for taking the FastAI course.
- Watch a lecture e2e one time.
- Copy each given notebook and follow through on your own.
- Think about main steps from the notebook and write those down (I usually use these main steps as the title of each section).
- Expand each of the main steps and break it down smaller steps so you can easily ask the copilot to write the code.
- Reproduce the results.
- Publish your notebook to github.


# A few key points from the lesson 05


* To me, the purpose of the chapter 5 is to show how to create a deep learning model from the scratch by setting up a linear model and a neural net first and then expanding them to create a deep learning model.
* We are simply using plain python and pytroch to create the model so that we can understand the pains of not using the framework such as FastAI.
 - In the chapter 4, we used the hugging face library to train the existing model. For the previous chapters, we used fastai. But here, we are only relying on python and pytorch (To be 100% correct, we use the fast ai library once to create a validation set. We learned about how to create a meaningful validation set from the scratch in [Iterate like a grandmaster](https://www.kaggle.com/code/jhoward/iterate-like-a-grandmaster/#Creating-a-validation-set).
- Spoiler: It is painful as we need to figure out how to do initialization, normalization and learning rate from the scratch. The cost of this pain is that it is making hard for people to experiment more.
- We will go through the same procdess but using the framework in our [second notebook](https://www.kaggle.com/code/jhoward/why-you-should-use-a-framework)

# Important Context
* The given problem we are going to use:
  - We are tasked to predict which passengers survived the Titanic shipwreck.
  - This is a very beginner frinedly kaggle competition. [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic/overview/evaluation).
  - You can find the data here: https://www.kaggle.com/competitions/titanic/data
  - The metric(evaluation) we are going to use is accuracy. You can find the information in the [evaluation](https://www.kaggle.com/competitions/titanic/overview/evaluation) tab.
* This is the same data we used for the [Lesson 03](https://course.fast.ai/Lessons/lesson3.html) where we created a neural net from the scratch in the excel.
  - You can find my notes for this lesson here: https://github.com/bigyunicorn/fastai22/blob/main/lesson03/titanic-data-visualization.ipynb



## Set up:
- Make sure your kernel is running on GPU.
- Get Kaggle API key from Kaggle (it will download kaggle.json).
- Copy the key value and put it in the right below cell.
- More details: https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners#Introduction

In [1]:
creds = '{"username":"<your kaggle user name>","key":"<your kaggle api key>"}'

In [3]:
# only need to run once.
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

Okay, now ready to use Kaggle API to download the dataset!

Let's first install the kaggle package.

## Step 1: Get the dataset from Kaggle

In [4]:
# install the kaggle package so we can use its API.
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# The below is useful code to fetch dataset we are interested in from Kaggle.
# you can use the same code if you are on kaggle.
import os
from pathlib import Path

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: path = Path('../input/titanic')
else:
    path = Path('titanic') #kaggle dataset data tab has this name at the end.
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(str(path)) # this is the kagglge python library API
        zipfile.ZipFile(f'{path}.zip').extractall(path) # extract all files from zip

100%|██████████| 34.1k/34.1k [00:00<00:00, 1.17MB/s]

Note: the below is a bsh command to download the data. Github copilot might recommend this line.

`!kaggle competitions download -c titanic`

What i just did above is loading the kaggle dataset to google colab. The reason I chose this is a) I prefer google colab over kaggle and b) I think i have flexibility to get more GPU if i need to.

## Step 2: Check the dataset structure and explore the data.

In [8]:
# Now, let's see the file structure of Kaggle downloaded dataset
!ls {path}

gender_submission.csv  test.csv  train.csv


Once I confirmed that the files are in csv, I am going to use pandas, as it is for handling csv files. Google Colab already installed pandas so I can skip `pip install pandas` and directly import the library as pd.

In [7]:
import pandas as pd # Colab has it installed already.